In [1]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
import pandas as pd
import spacy
from spacy import displacy

nltk.download('stopwords')

2022-05-31 12:45:43.098285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-31 12:45:43.098306: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package stopwords to /home/nguyen/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# !python3 -m spacy download en_core_web_sm
# !python3 -m spacy download en_core_web_md
!python3 -m spacy download en_core_web_lg

nlp = spacy.load('en_core_web_lg')

# text = '''Given the recent downturn in stocks especially in tech which is likely to persist as yields keep going up, I thought it would be prudent to share the risks of investing in ARK ETFs, written up very nicely by [The Bear Cave](https://...). The risks comes primarily from ARK's illiquid and very large holdings in small cap companies. ARK is forced to sell its holdings whenever its liquid ETF gets hit with outflows as is especially the case in market downturns. This could force very painful liquidations at unfavorable prices and the ensuing crash goes into a positive feedback loop leading into a death spiral enticing even more outflows and predatory shorts.'''

2022-05-31 12:45:49.701886: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-31 12:45:49.701906: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [15]:
# DATASET = './datasets/allnews.csv'
# DATASET = './datasets/headlines_5_26_2022_11_15.csv'
# DATASET = './datasets/headlines.csv'
# DATASET = '/home/nguyen/Desktop/news_spider_project/src/service/news_spider/news_spider/spiders/output/dataset/headlines/CNN_5_31_2022/headlines.csv'
DATASET = '/home/nguyen/Desktop/news_spider_project/src/service/news_spider/news_spider/tmp/marketwatch/headlines_5_31_2022.csv'
IGNORE_ENTITIES = './datasets/ignore_entity.txt'
COUNTER_FILE = './counter.txt'

In [16]:
def count_entities(text, counter = None, debug = False, 
    interested_labels = ['ORG', 'PERSON'], 
    ignore_entities = ['Reuters', 'CNN','MarketWatch']):

    doc = nlp(text)

    if debug:
        displacy.render(doc, style = 'ent')

    if counter == None:
        counter = Counter()

    # Only count the entity once for every article
    seen_entity = Counter()

    for entity in doc.ents:
        if entity.label_ in interested_labels and entity.text not in ignore_entities and entity.text not in seen_entity:
            counter[entity.text] +=1
            seen_entity[entity.text] +=1
    return counter


In [17]:
df = pd.read_csv(DATASET)
# df = df.dropna()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  25 non-null     int64 
 1   title       25 non-null     object
 2   date        1 non-null      object
 3   text        25 non-null     object
 4   authors     25 non-null     object
 5   source      25 non-null     object
 6   url         25 non-null     object
 7   image_url   25 non-null     object
dtypes: int64(1), object(7)
memory usage: 1.7+ KB


In [18]:
df.head()


,Unnamed: 0,title,date,text,authors,source,url,image_url
0,0,Brace for another selloff if U.S. stocks reach...,NaN,“Top Gun: Maverick” is taking American cinemas...,['Joseph Adinolfi'],MarketWatch,https://www.marketwatch.com/story/brace-for-an...,https://images.mktw.net/im-554330/social
1,1,Big talk and big money: Texas con duped dozens...,NaN,The swindle always began with a flashy show of...,['Lukas I. Alpert'],MarketWatch,https://www.marketwatch.com/story/big-talk-and...,https://images.mktw.net/im-554228/social
2,2,"Canada plans to ban purchase, sale of handguns...",NaN,OTTAWA — Canada said on Monday that it is intr...,['Paul Vieira'],MarketWatch,https://www.marketwatch.com/story/canada-plans...,https://images.mktw.net/im-554190/social
3,3,Clinton campaign lawyer Sussmann acquitted in ...,NaN,WASHINGTON (AP) — A lawyer for Hillary Clinton...,['Associated Press'],MarketWatch,https://www.marketwatch.com/story/clinton-camp...,https://images.mktw.net/im-554564/social
4,4,Excitement in Shanghai as officials promise re...,NaN,There was excitement in Shanghai Tuesday after...,['Ciara Linnane'],MarketWatch,https://www.marketwatch.com/story/excitement-i...,https://images.mktw.net/im-554407/social


In [19]:
print('rows = ', len(df['title']))

ignore = list()
with open(IGNORE_ENTITIES, 'r') as file:
    for line in file.readlines():
        ignore.append(line.strip())

print(ignore)

counter = Counter()

counter = count_entities(df['text'][10], counter = counter)

for index, row in df.iterrows():
    if row['text'] == 'NaN' or row['text'] == '':
        print(f"Empty text content, skipping this {row['title']}")
        continue

    print(f"extracting index: {index}, {row['title']}")
    try:
        counter = count_entities(row['text'], counter = counter, ignore_entities=ignore)
    except Exception as e:
        print('ERROR ROW ', row)
        print(e)
        break
    # break

print(counter)

rows =  25
['CNN Business', 'Reuters', 'CNN', 'MarketWatch']
extracting index: 0, Brace for another selloff if U.S. stocks reach this technical 'danger zone'
extracting index: 1, Big talk and big money: Texas con duped dozens into believing they were investing in events like a Rihanna tour and the Mayweather-Pacquiao fight
extracting index: 2, Canada plans to ban purchase, sale of handguns: ‘There is no reason anyone in Canada should need guns in their everyday lives’
extracting index: 3, Clinton campaign lawyer Sussmann acquitted in setback to Durham probe of origins of Trump-Russia investigation
extracting index: 4, Excitement in Shanghai as officials promise reopening after two-month lockdown, and cases continue to flatten in U.S. Northeast
extracting index: 5, What GM’s idea for an EV with two charging ports says about the car market
extracting index: 6, Why Is the Stock Market Down Today? Dow Slips, Oil Prices Rise, Alibaba Jumps.
extracting index: 7, Enservco Shares Rise 13%
extr

In [12]:
with open(COUNTER_FILE, 'w') as file:
    for key, value in counter.most_common():
        print(key, value)
        if value > 2:
            file.write(f"{key}:{value}\n")

Vladimir Putin 5
New York 5
Microsoft 4
the Labor Department 4
XYZ 4
Twitter 4
Boeing 4
NASA 4
Putin 3
the European Union 3
Google 3
Freddie Mac 3
Tesla 3
Facebook 3
Meta 3
Apple 3
Cameron Spencer/Getty Images 3
Harmony 3
United Launch Alliance Atlas V 3
CST-100 Starliner 3
OFT-2 3
Starliner 3
Commercial Crew Program 3
Joel Kowsky 3
SpaceX 3
Amazon 3
Moon 3
Kremlin 2
William Jackson 2
Capital Economics 2
Timothy Ash 2
Bluebay Asset Management 2
The International Monetary Fund 2
Alexander Novak 2
state news agency RIA 2
Novak 2
the World Economic Forum 2
Xi Jinping 2
Xi 2
crypto 2
digital 2
Li Keqiang 2
Li 2
UBS 2
Netflix 2
Mercedes 2
Jones 2
Congress 2
Realtor.com 2
the International Space Station 2
CARL DE SOUZA 2
CARL DE SOUZA/ 2
Getty Images 2
Apollo 2
CNN Business 1
Volodymyr Zelensky 1
solidarity 1
Jason Furman 1
Harvard 1
Barack Obama's 1
Viktor Orban 1
the European Union's 1
Joe Biden 1
George Soros 1
Sergei 1
Yulia Skripal 1
Eduard Heger 1
Soros 1
Mujtaba Rahman 1
the Eurasia G